In [1]:
import cv2
import numpy as np


In [2]:
img = cv2.imread("sudoku.jpg", 1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(gray, (5, 5), 0)

thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 2)

edged = cv2.Canny(gray, 80, 200)
edged = cv2.bitwise_not(edged)


### DEBUG ###

cv2.imshow("orignal", img)
cv2.imshow("gray", gray)
cv2.imshow("blur", blur)
cv2.imshow("threshold", thresh)
cv2.imshow("edged", edged)

###

cv2.waitKey(0)
cv2.destroyAllWindows()

## Finding Contours

You can use either thresh or edged, both should work just fine

In [3]:
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

cont = img.copy()
cv2.drawContours(cont, contours, -1, (0, 0, 255), 2)

cv2.imshow("contour", cont)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Finding the outer box

In [30]:
outer_cnt = None
max_area = 0

for cnt in contours:
    
    cur_area = cv2.contourArea(cnt)
    
    if 25000 < cur_area < 200000 and max_area < cur_area:
        
        perimeter = cv2.arcLength(cnt, True)
        simplified = cv2.approxPolyDP(cnt, 0.01*perimeter, True)
    
        if len(simplified) == 4:
            outer_cnt = simplified
            max_area = cur_area

            
if outer_cnt is not None:
    
    cont = img.copy()
    cv2.drawContours(cont, [outer_cnt], -1, (0, 255, 0), 15)
    
    cv2.imshow("contour", cont)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
        
        

print(outer_cnt.shape)

(4, 1, 2)


## Finding the smaller boxes

In [31]:
smaller = []

for cnt in contours:
    
    area = cv2.contourArea(cnt)
    
    if 400 < area < 3000:
        
        perimeter = cv2.arcLength(cnt, True)
        simplified = cv2.approxPolyDP(cnt, 0.01*perimeter, True)
    
        if len(simplified) == 4:
            smaller.append(cnt)

img_copy = img.copy()
cv2.drawContours(img_copy, smaller, -1, (0, 255, 0), 3)
cv2.imshow("smaller boxes", img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
## lr means sort left to right
## tb means sort top to bottom
## rowise means left to right first, then top to bottom

def sort_contours(cnts, lr=True, tb=True, rowise=True):
    
    if rowise:
        key = lambda x: (x[1][1] if tb else -x[1][1], x[1][0] if lr else -x[1][0])
    else:
        key = lambda x: (x[1][0] if lr else -x[1][0], x[1][1] if tb else -x[1][1])


    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes), key=key))
    
    return (cnts, boundingBoxes)
    

In [23]:
smaller, boundingBoxes = sort_contours(smaller)
print(len(boundingBoxes))

### DEBUG: check if sort working properly ###

img_copy = img.copy()
cv2.drawContours(img_copy, smaller, 9, (0, 255, 0), 3)
# cv2.drawContours(img_copy, smaller, 2, (0, 255, 0), 3)
cv2.imshow("2nd & 9th box sorted", img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()

### DEBUG ###

81


In [38]:
boundingBoxes = np.array(boundingBoxes, dtype='object').reshape(-1, 9, 4)
boundingBoxes.shape

(9, 9, 4)

In [35]:
import time
rois = []

for i in boundingBoxes:
    rois.append([])
    for j in i:
        rois[-1].append(img[j[1]:j[1]+j[3], j[0]:j[0]+j[2]])
        
        cv2.imshow(f"roi-{time.time()}", rois[-1][-1])
        
cv2.waitKey(0)
cv2.destroyAllWindows()